# Comparing Regressors
This notebook compares multiple regression methods on multiple datasets and evaluates them in terms of the $r^2$-measure.

__Note:__ There is a similar [notebook for classification datasets](Comparing_Classifiers.ipynb).

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

try:
    # Model Trees are installed / on the path
    from modeltrees import ModelTreeRegressor
except:
    # Assume project structure
    import sys
    sys.path.append("..")
    from modeltrees import ModelTreeRegressor
from modeltrees.criteria import SumOfSquareErrorSplitCriterion

import pandas as pd
import numpy as np

# Downloading and accessing files
import shutil
import urllib.request
from urllib.parse import urlparse
from pathlib import Path
import os


## 1. Datasets
In this section, all datasets for the comparison are defined. Missing datasets are downloaded automatically.

See [Section 3.3](#characteristics) for a list of dataset characteristics

### 1.1 Downloading Datasets
We do not ship datasets with this repository, but the notebook will automatically try to download missing data.

In [2]:
data_path = "./data"

def get_dataset_file_path(dataset_id, url, file=None):
    # If no file is specified, take the name from the url
    if file is None:
        file = urlparse(url)
        file = os.path.basename(file.path)
     
    # Create path to local file
    path = Path(data_path, dataset_id, file)
    
    if not path.exists():
        # Create missing folders
        os.makedirs(path.parent, exist_ok=True)
        
        # Download missing file
        with urllib.request.urlopen(url) as response:
            with open(path, "wb") as outputFile:
                shutil.copyfileobj(response, outputFile)
    
    return path

### 1.2 Dataset Definitions
The following defines different datasets with their download url, and possibly some preprocessing steps

In [3]:
def fetch_beijing_pm25():
    # Dataset name and source
    ds_name = "beijing_pm25"
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv"
    
    # Load csv file
    path = get_dataset_file_path(ds_name, url)
    df = pd.read_csv(path, index_col=0)
    
    # Filter NAN and inf in target variable
    df = df[np.isfinite(df["pm2.5"])]
    
    # Get Target variable
    y = df["pm2.5"].values
    df.drop(columns="pm2.5", inplace=True)
    
    # Preprocess Features
    #    One-Hot Encoding of categorical variable
    df = pd.get_dummies(df, columns=["cbwd"])
    
    # Get Features
    X = df.values
    
    return X, y, {'ref':'https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data'}

### 1.3 Iterating over all Datasets
This gives a generator that iterates over all datasets.  
Each dataset is a triple consisting of 
- Features Matrix `X`, 
- Target Vector `y`, and 
- An attribute dictionary that contains meta information like the name of the dataset or a reference url

In [4]:
def get_datasets():
    # Using generators instead of lists for memory efficiency reasons.
    
    # Dataset 1: California Housing
    data = fetch_california_housing()
    X = data.data
    y = data.target
    attr = {       # Attributes
        'name': 'Cal. Housing',
        'ref': 'https://scikit-learn.org/stable/datasets/index.html#california-housing-dataset'
    }   
    
    yield (X, y, attr)
    
    # Dataset 2: Beijing PM2.5
    X, y, attr = fetch_beijing_pm25()
    attr['name'] = 'Beijing PM2.5'
    yield (X, y, attr)

## 2. Regressors
We are comparing the following regressors:
- Linear Regression
- Decision Trees with maximal depth 3 and 6 
- Model Trees with maximal depth 1 and 3. We compare two split criteria:
    - Plain Gradients 
    - Renormalized Gradients

In [5]:
def get_regressors():
    # Create custom criterion
    sse = SumOfSquareErrorSplitCriterion()
    return [
        (LinearRegression(normalize = True), "Lin. Reg."),
        (DecisionTreeRegressor(max_depth=3, random_state=12), "Decision Tree"),
        (DecisionTreeRegressor(max_depth=6, random_state=12), "Decision Tree"),
        (ModelTreeRegressor(max_depth=1, criterion=sse), "Model Tree (SSE)"),
        (ModelTreeRegressor(max_depth=3, criterion=sse), "Model Tree (SSE)"),
        (ModelTreeRegressor(max_depth=1), "Model Tree (Gradient)"),
        (ModelTreeRegressor(max_depth=3), "Model Tree (Gradient)"),
        (ModelTreeRegressor(max_depth=1, criterion="gradient-renorm-z"), "Model Tree (Renorm. Gradient)"),
        (ModelTreeRegressor(max_depth=3, criterion="gradient-renorm-z"), "Model Tree (Renorm. Gradient)")
    ]

## 3. Comparison
### 3.1 Parameters

In [6]:
# Cross Validation: Number of Folds
n_fold = 5

seed = 42   # We suggest to try other values to get a feeling for the stability

### 3.2 Evaluation
Iterating over datasets and regressors and evaluating the regressors in terms of the $r^2$ metric.

In [7]:
# Create a DataFrame for results (see 3.4)
# Multi-Index for better readability
col_index = pd.MultiIndex(levels=[[],[]],
                             codes=[[],[]],
                             names=['Method', 'Depth'])
results = pd.DataFrame(columns=col_index)

# Create a DataFrame for the Dataset Characteristics (see 3.3)
ds_characteristics = pd.DataFrame(columns=("#Samples", "#Features", "Reference"))

# Create a scorer function
scorer = make_scorer(r2_score)

# Iterate over Datasets
for X, y, attr in get_datasets():
    ds_name = attr['name']
    
    # Store dataset  characteristics
    n_samples, n_features = X.shape
    ds_characteristics.loc[ds_name, "#Samples"] = n_samples
    ds_characteristics.loc[ds_name, "#Features"] = n_features
    
    if "ref" in attr:
        ds_characteristics.loc[ds_name, "Reference"] = attr["ref"]
    else:
        ds_characteristics.loc[ds_name, "Reference"] = None
    
    # Iterate over Regressors
    for model, m_name in get_regressors():
        # Use the same seed for comparing different regressors
        kfold = KFold(n_splits=n_fold, shuffle=True, random_state=seed)
        
        # Compute scores of all k folds (see variable n_fold)
        scores = cross_val_score(model, X, y, scoring=scorer, cv=kfold)
        
        # Compute statistics from list of scores
        mean_score = np.mean(scores)
        std_score = np.std(scores)
        
        # Create result cell
        cell_text = f"{mean_score*100:.2f} ± {std_score*100:.2f}" 
        
        # Multi-Indexing
        if hasattr(model, 'max_depth'):
            col_idx = (m_name, f"{model.max_depth}")
        else:
            col_idx = (m_name, '-')
            
        results.loc[ds_name, col_idx] = cell_text

### 3.3 Dataset Characteristics <a id='characteristics'></a>

In [8]:
ds_characteristics["#Samples"] = ds_characteristics["#Samples"].astype(dtype=np.int)
ds_characteristics["#Features"] = ds_characteristics["#Features"].astype(dtype=np.int)

def format_link(val):
    # Handle Empty references
    if val is None:
        return ''
    
    # Format link
    return '<a target="_blank" href="{}">Link</a>'.format(val)

ds_characteristics.style.format({'Reference': format_link})

### 3.4 Results
The regressors are evaluated in terms of the $r^2$ metric.  
The following results are given in percentage. The uncertainty is given as standard deviation of the $r^2$ score.

In [9]:
results

Method            Lin. Reg. Decision Tree               Model Tree (SSE)  \
Depth                     -             3             6                1   
Cal. Housing   60.14 ± 1.70  52.70 ± 1.46  64.15 ± 1.29    49.07 ± 24.42   
Beijing PM2.5  27.48 ± 0.52  21.63 ± 1.15  38.83 ± 0.95     31.62 ± 0.82   

Method                      Model Tree (Gradient)                \
Depth                     3                     1             3   
Cal. Housing   62.19 ± 9.31          67.09 ± 1.57  68.32 ± 1.68   
Beijing PM2.5  37.20 ± 0.89          31.38 ± 0.84  47.04 ± 1.42   

Method        Model Tree (Renorm. Gradient)                
Depth                                     1             3  
Cal. Housing                   67.16 ± 1.50  72.24 ± 1.05  
Beijing PM2.5                  35.04 ± 0.71  50.78 ± 1.09